# VERGE Batch Processing 2:

Get the initial embeddings for a collection of tiles.

## Processing Setup

In [1]:
# Google colab
import os
from google.colab import drive
drive.mount('/content/drive')
project_home = '/content/drive/MyDrive/Projects/verge'
os.chdir(project_home)
!pip install geo_encodings osmnx

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.6 MB/s eta 0:00:00


In [2]:
# Local processing setup
# project_home = '..'

## Notebook Setup

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from typing import List, Tuple, Optional

import pickle
import json
import copy
import pandas as pd
import numpy as np
import pyproj
import shapely
import osmnx
import geopandas

import sys
sys.path.append('%s/03-embeddings' % project_home)
from embedderv5 import *

sys.path.append(project_home)
from utils.verge import rules


## Parameters

In [4]:
# The name of the ROI to use.
roi_name = 'newengland'

# The name of the general-purpose data directory.
data_home = '%s/data' % (project_home)

# The name of the ROI-specific data directory.
roi_home = '%s/data/%s' % (project_home, roi_name)

# The unique identifier of the model to be used.
transformer_run_id = '301b'
collector_run_id = '301b'

## Preliminaries

In [5]:
# Read the ROI definition.
fname = '%s/roi.json' % roi_home
with open(fname) as source:
    roi = json.load(source)

tile_size = roi['tile_size']
encoding_resolution = roi['encoding_resolution']

roi

{'name': 'newengland',
 'lon0': -73.564321,
 'lat0': 41.253746,
 'lon1': -68.058533,
 'lat1': 45.116468,
 'proj_def': '\n+proj=tmerc +lat_0=43.185107 +lon_0=-70.81142700000001\n+k=1.0 +x_0=231000.0 +y_0=211000.0 +datum=WGS84 +units=m +no_defs\n',
 'tile_size': 2000,
 'tile_shift': 1000,
 'encoding_resolution': 100}

In [6]:
# Re-define the ROI. The original is taking too long.
# this just includes the more urbanized part os sourhtern New England
roi['lat0'] =  41.2
roi['lon0'] = -73.4

# roi['lat1'] = 43.4
# roi['lon1'] = -69.7

# even smaller:
roi['lat1'] = 42.0
roi['lon1'] = -72.3


In [7]:
# Read the file containing labels.
fname = '%s/labels.csv' % data_home
labels = pd.read_csv(fname)

# Make a lookup table to get a numerical label from a text label.
label_lookup = {
    z['label']: z['id']
    for z in labels.to_dict('records')
}
label_count = len(label_lookup)
label_lookup

{'amenity : commercial': 0,
 'amenity : food and drink': 1,
 'amenity : parking lot': 2,
 'amenity : recreation': 3,
 'landuse : agricultural': 4,
 'landuse : commercial': 5,
 'landuse : forest': 6,
 'landuse : industrial': 7,
 'landuse : meadow': 8,
 'landuse : recreation': 9,
 'landuse : residential': 10,
 'landuse : retail': 11,
 'railway : rail': 12,
 'railway : rail stop': 13,
 'route : highway': 14,
 'route : primary road': 15,
 'route : residential road': 16,
 'route : secondary road': 17,
 'route : tertiary road': 18,
 'waterway : lakes and ponds': 19,
 'waterway : land': 20,
 'waterway : rivers and streams': 21}

In [8]:
# Define a local map projection, using the definition from the ROI file.
def get_projections(proj_def):
    ltm_crs = pyproj.CRS.from_proj4(proj_def)
    wgs84_crs = pyproj.CRS.from_epsg(4326)
    proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
    proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform
    return proj_forward, proj_inverse

proj_forward, proj_inverse = get_projections(roi['proj_def'])

In [9]:
# # Read the coastline file.
# fname = '%s/coastlines' % (roi_home)
# coastlines_gdf = geopandas.read_file(fname)
# print('%d coastline polygons' % len(coastlines_gdf))

# def get_land_water(bounds, features):

#     # Create a baseline polygon consisting of the whole AOI.
#     landwater = copy.deepcopy(bounds)

#     # Intersect that with the coastlines data.
#     coastlines = shapely.union_all(coastlines_gdf['geometry'].values)
#     landwater = landwater.intersection(coastlines)

#     # subtract out any polygonal water feature.
#     for _, f in features.iterrows():
#         if f['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             if f['natural'] == 'water':
#                 landwater = shapely.difference(landwater, f['geometry'])

#     return landwater

## Processing


### Prepare the list of geospatial entities

In [10]:
import glob
globstring = '%s/batch/gents/*' % roi_home
fnames = glob.glob(globstring)
print('%d geospatial entity files' % len(fnames))

# Read each of the CSV files named in the list "fnames".
# Concatenate all records into one data frame named "gents",
# de-duplicating based on the "id" field.
gents_list = []
for fname in fnames:
    df = pd.read_csv(fname)
    gents_list.append(df)

gents = pd.concat(gents_list, ignore_index=True)
print(len(gents))

gents = gents.drop_duplicates(subset=['id'])
print('%d geospatial entities after de-duplication' % len(gents))

# Create a real geometry field and use it to create a spatial index.
gents['geometry'] = gents['geomxy'].apply(shapely.wkt.loads)
gents = geopandas.GeoDataFrame(gents, geometry='geometry')
gents_sindex = gents.sindex

197 geospatial entity files


EmptyDataError: No columns to parse from file

### Prepare the list of tiles

In [ ]:
# Get a starting x,y in projected coordinates.
x0, y0 = proj_forward(roi['lon0'], roi['lat0'])
print(x0, y0)

buffer = 3000
x0 = x0 + buffer
x0 = np.ceil(x0 / roi['tile_size']) * roi['tile_size']
y0 = y0 + buffer
y0 = np.ceil(y0 / roi['tile_size']) * roi['tile_size']
print(x0, y0)

x1, y1 = proj_forward(roi['lon1'], roi['lat1'])
x1 = x1 - buffer
x1 = np.floor(x1 / roi['tile_size']) * roi['tile_size']
y1 = y1 - buffer
y1 = np.floor(y1 / roi['tile_size']) * roi['tile_size']
print(x1, y1)

tile_list = []
for x in range(int(x0), int(x1), roi['tile_size']):
    for y in range(int(y0), int(y1), roi['tile_size']):
        tile_list.append((x, y))

print('%d tiles' % len(tile_list))


In [ ]:
from geo_encodings import MPPEncoder
encoder = MPPEncoder(
    region=[0, 0, roi['tile_size'], roi['tile_size']],
    resolution=roi['encoding_resolution'],
    center=True
)
geo_encoding_dim = len(encoder)
geo_encoding_dim

In [ ]:
# Loop over the tiles in "tile_list".
tile_encodings = []

for k, tile in enumerate(tile_list):

    if k % 100 == 0:
    print('Processing tile %d / %d' % (k, len(tile_list)))

    tile_x, tile_y = tile

    # Define a bounding box for this tile in projected coordinates.
    x0, y0 = tile_x, tile_y
    x1, y1 = tile_x + roi['tile_size'], tile_y + roi['tile_size']
    xx = [x0, x1, x1, x0, x0]
    yy = [y0, y0, y1, y1, y0]
    tile_bbox = shapely.Polygon(list(zip(xx, yy)))

    # Extract all geospatial entities from "gents" using "gents_sindex".
    possible_matches_index = list(gents_sindex.intersection(tile_bbox.bounds))
    possible_matches = gents.iloc[possible_matches_index]
    intersecting_gents = possible_matches[possible_matches.intersects(tile_bbox)].copy()

    # Clip any intersecting entities to the tile, and shift their coordinates to be relative to the tile lower left corner.
    tile_gents = []
    for _, gent in intersecting_gents.iterrows():
        geomxy = gent['geometry'].intersection(tile_bbox)
        if geomxy.is_empty:
            continue
        tile_gents.append({
            'category': gent['category'],
            'label': gent['label'],
            'geometry': shapely.affinity.translate(geomxy, xoff=-x0, yoff=-y0),
            'gtype': gent['gtype'],
            'xoff': x0,
            'yoff': y0,
        })

    if not tile_gents:
        continue

    # Apply MPP encoding to each.

    for gent in tile_gents:
        gent['encoding'] = encoder.encode(gent['geometry']).values()

    # Compute a one-hot label vector for each entity.
    label_count = len(label_lookup)
    for gent in tile_gents:
        label_name = '%s : %s' % (gent['category'], gent['label'])
        label_id = label_lookup[label_name]
        label_onehot = np.full(label_count, 0, dtype=float)
        label_onehot[label_id] = 1
        gent['onehot'] = label_onehot

    # Concatenate the one-hot vectors and the MPP encodings for each matrix.
    mpps = np.vstack([z['encoding'] for z in tile_gents])
    onehots = np.vstack([z['onehot'] for z in tile_gents])
    tile_encoding = np.hstack([onehots, mpps])

    tile_encodings.append({
        'x': tile_x,
        'y': tile_y,
        'encoding': tile_encoding
    })

print(f'Generated encodings for {len(tile_encodings)} tiles.')

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# data = e
# plt.imshow(data, cmap='viridis', aspect='equal')  # 'viridis' is a good default colormap
# plt.colorbar(label="Value")  # add a color scale bar
# plt.title("Full Encoding For Tile")
# plt.xlabel("Encoding Index")
# plt.ylabel("Entity Number")
# plt.show()


### Initial embedding for this tile

In [ ]:
# Load the initial embedding model.
from utils.geo_transformer_mem import VergeDataset, verge_collate_fn, GeospatialTransformer

transformer = GeospatialTransformer(
    feature_dim = geo_encoding_dim + label_count,
    model_dim=128,
    num_heads=4,
    num_layers=4,
    num_classes=label_count,
    dropout=0.2
)

model_fname = '%s/models/transformer-%s' % (roi_home, transformer_run_id)
transformer = torch.load(model_fname, weights_only=False, map_location=torch.device('cpu'))
print('loaded %s' % model_fname)

n_param = sum(p.numel() for p in transformer.parameters() if p.requires_grad)
print('%d trainable parameters in model' % n_param)


In [ ]:
initials = []
for enc in tile_encodings:
    tile_x = enc['x']
    tile_y = enc['y']
    encoding = enc['encoding']
    input_features = torch.tensor(encoding, dtype=torch.float32).unsqueeze(0)
    input_attention_mask = torch.ones(1, encoding.shape[0], dtype=torch.float32)
    transformed = transformer.embed(input_features, input_attention_mask)
    initials.append({
        'x': tile_x,
        'y': tile_y,
        'initials': transformed
    })

print(f'Generated initial encodings for {len(initials)} tiles.')

### Get the final embedding for this tile

In [ ]:
import sys
sys.path.append('%s/03-embeddings' % project_home)
from embedderv5 import ContrastivePairDataset, PermutationInvariantModel, TripletContrastiveLoss, triplet_collate_fn

In [ ]:
# Initialize model
embedding_dim = 128
model = PermutationInvariantModel(
    input_dim=embedding_dim,
    hidden_dim=128,
    embedding_dim=embedding_dim,
    num_attention_heads=8,
    num_linear_layers=3,
    dropout=0.1
)

model_fname = '%s/models/collector-%s.pth' % (roi_home, collector_run_id)
state_dict = torch.load(model_fname, map_location='cpu')
model.load_state_dict(state_dict)
print('loaded %s' % model_fname)

n_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('%d trainable parameters in model' % n_param)


In [ ]:
finals = []
for item in initials:
    initial_encoding = item['initials']
    # Create a mask of ones with the correct shape
    mask = torch.ones(initial_encoding.shape[0], initial_encoding.shape[1], dtype=torch.bool)
    final_embedding = model(initial_encoding, mask)
    finals.append({
        'x': item['x'],
        'y': item['y'],
        'embedding': final_embedding
    })

print(f'Generated final embeddings for {len(finals)} tiles.')

In [ ]:
final_embeddings_with_bbox = []
for item in finals:
    tile_x = item['x']
    tile_y = item['y']
    embedding = item['embedding']

    # Compute the projected bounding box.
    x0, y0 = tile_x, tile_y
    x1, y1 = tile_x + roi['tile_size'], tile_y + roi['tile_size']

    # Compute the lon/lat bounding box using the inverse projection.
    lon0, lat0 = proj_inverse(x0, y0)
    lon1, lat1 = proj_inverse(x1, y1)

    final_embeddings_with_bbox.append({
        'x': tile_x,
        'y': tile_y,
        'lon0': lon0,
        'lat0': lat0,
        'lon1': lon1,
        'lat1': lat1,
        'center_lon': (lon0 + lon1) / 2,
        'center_lat': (lat0 + lat1) / 2,
        'embedding': embedding.detach().numpy()
    })

print(f'Added bounding boxes to {len(final_embeddings_with_bbox)} final embeddings.')

In [ ]:
# Save the final embeddings.
output_dir = '%s/batch' % roi_home
os.makedirs(output_dir, exist_ok=True)
output_fname = '%s/final_embeddings.pkl' % output_dir

with open(output_fname, 'wb') as f:
    pickle.dump(final_embeddings_with_bbox, f)

print(f'Saved final embeddings to {output_fname}')

In [ ]:
final_embeddings_with_bbox[0]